1. Import functions

In [6]:
import os
import warnings
from collections import deque
from datetime import timedelta
import random
import numpy as np
import pandas as pd
from scipy.sparse import bmat, coo_matrix, csr_matrix, diags, kron, load_npz
from scipy import linalg
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.constraints import NonNeg
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, TimeDistributed, Flatten, Reshape
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Model
from tabulate import tabulate
from itertools import chain
from termcolor import colored
from random import randrange
from IPython.display import clear_output
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import cupy as cp
import time
import traceback
import datetime
import math
from livelossplot import PlotLossesKeras

2. Check GPU

In [7]:
numpy_array = np.random.rand(100)
cupy_array = cp.asarray(numpy_array)
if cupy_array.data.device.id >= 0:
    print("CuPy is using the GPU.")
else:
    print("CuPy is NOT using the GPU.")

def unfolding_3D(Tens, unfol_dim, other_dim_seq ):
    X = Tens.transpose(unfol_dim, other_dim_seq[0], other_dim_seq[1]).reshape(Tens.shape[unfol_dim], Tens.shape[other_dim_seq[0]] * Tens.shape[other_dim_seq[1]])
    return (X)
def folding_3D(Unfold_Tens, unfol_dim , other_dim_seq, Tens_shape):
    a = [0,1,2]
    items = deque(a)
    items.rotate(unfol_dim) 
    a_up= list(items)
    X = Unfold_Tens.reshape(Tens_shape[unfol_dim], Tens_shape[other_dim_seq[0]],Tens_shape[other_dim_seq[1]]).transpose(a_up[0], a_up[1], a_up[2])    
    return (X)

CuPy is using the GPU.


In [8]:
import os
name= 'Logan'
Data_directory =os.getcwd()

#Data_directory = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\Datasets'
from scipy import sparse
Tens = np.load(Data_directory + os.sep + 'Train_' + name + '_3D_data.npy')
Test_Tens = np.load(Data_directory + os.sep + 'Test_' + name + '_3D_data.npy')
test_dataset = pd.read_csv(Data_directory + os.sep + 'test_' + name + '_data.csv', index_col = [0,1], header = [0])
Val_Tens = np.load(Data_directory + os.sep + 'Val_' + name + '_3D_data.npy')
val_dataset = pd.read_csv(Data_directory + os.sep + 'Val_' + name + '_data.csv', index_col = [0,1], header = [0])
ind = pd.read_csv(Data_directory + os.sep + 'date_time_train.csv', index_col = [0,1])
stl_clm = pd.read_csv(Data_directory + os.sep + 'section_names.csv', index_col = 0)
train_data_shubham= pd.read_csv(Data_directory + os.sep + r'Train_' + name + '_data.csv', index_col = [0,1], header = [0])
W = sparse.load_npz(Data_directory + os.sep + 'W'+ '.npz')
#W = np.load(Data_directory + os.sep + 'Static_Adjacency_'+ name + '_Topo_Traffic.npy')
MWY_Sections = pd.read_csv(Data_directory + os.sep + r'MWY_Sections.csv')
section_ids = MWY_Sections['Sections'].tolist()  # Get the list of section IDs'''
section_ids = train_data_shubham.columns.to_list()
serial_numbers = [test_dataset.columns.get_loc(section_id)  for section_id in section_ids]
Tens = Tens[serial_numbers, :, :]
Test_Tens = Test_Tens[serial_numbers, :, :]
test_dataset = pd.read_csv(Data_directory + os.sep + 'test_' + name + '_data.csv', index_col = [0,1], header = [0])
test_dataset = test_dataset.T.iloc[serial_numbers].T
Val_Tens = Val_Tens[serial_numbers, :, :]
val_dataset = pd.read_csv(Data_directory + os.sep + 'Val_' + name + '_data.csv', index_col = [0,1], header = [0])
val_dataset = val_dataset.T.iloc[serial_numbers].T
stl_clm = stl_clm.iloc[serial_numbers]
train_data_shubham = train_data_shubham.T.iloc[serial_numbers].T
stl_clm.index = stl_clm.index.astype('str')
test_dataset = test_dataset[stl_clm.index].T
test_data_prior = test_dataset.copy(deep = True)
val_dataset = val_dataset[stl_clm.index].T
val_data_prior = val_dataset.copy(deep = True)
test_dataset_col =  test_dataset.columns
val_dataset_col =  val_dataset.columns
train_data_shubham = train_data_shubham[test_data_prior.index.astype(str)]
train_data_shubham_prior = train_data_shubham.T
serial_numbers_W = serial_numbers.copy()
for i in range(1, 96):
    for j in serial_numbers:
        serial_numbers_W.append(i*476 + j)
#W= coo_matrix(W.toarray()[serial_numbers_W].T[serial_numbers_W].T)
W = coo_matrix(W)
W_dense = W.toarray()
'''# Create a heatmap plot of the dense matrix
plt.imshow(W_dense, cmap='hot', interpolation='nearest')
plt.colorbar()  # Add colorbar for reference
plt.title("Heatmap of W")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()'''
W = (W + W.T)/2
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
file_path = Data_directory + os.sep + 'Complete_Logan_3D_data.npy'
Complete_Tens = np.load(file_path)
Complete_Tens = Complete_Tens[serial_numbers, :, :]
Tens1 = Tens.transpose(0,2,1)

In [9]:
#Attention layer
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return tf.keras.backend.get_value(optimizer.lr)
    return lr

class TimingCallback(tf.keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)      

from tensorflow.keras.optimizers.legacy import Adam


In [5]:
A = np.load('A.npy')
B = np.load('B.npy')
C = np.load('C.npy')
rank = r  = A.shape[1]
mag_A = np.diag(np.asarray(np.sqrt((A.T.dot(A)).sum(axis=1))).reshape(-1))
mag_A_inverse = np.linalg.inv(mag_A)
data = pd.DataFrame((mag_A).dot(linalg.khatri_rao(B, C).T).T, index=ind.index).round(5)
data_max = np.array(data).max()
rank = r = A.shape[1]
rank

20

In [16]:
# from keras import backend as k
rolling_window1 = 12
forecast_window = 4
daily_points = 96
lstm_units = 100
#rnn_units = 400
lstm_act = 'elu'
rnn_act = 'elu'
return_seq = True
lstm_kernel_type= rnn_kernel_type = 'random_normal'
bi_layers = 2
dense_units = rank
dense_act = 'elu'
dropout_percent = 0.5
batchs = 68 * 100
Epochs = 800

In [17]:
from livelossplot import PlotLossesKeras
optim = Adam(learning_rate=.005)
lr_metric = get_lr_metric(optim)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=500)
cb = TimingCallback()
clbcks = [es,cb, PlotLossesKeras(from_step=1)]
loss_metrics = [RootMeanSquaredError(name='rmse'),MeanAbsolutePercentageError(name='mape'),get_lr_metric(optim)]

In [18]:
yTrain1final_g2 = cp.load('yTrain1final_g2.npy', )
xTrain1final_g2 = cp.load('xTrain1final_g2.npy', )
yVal1final_g1 = cp.load('yVal1final_g1.npy', )
xVal1final_g1 = cp.load('xVal1final_g1.npy', )
Index_final = cp.load('Index_final.npy', )
yTrain1true_Original1 = cp.load('yTrain1true_Original1.npy', )
Index_final_val = cp.load('Index_final_val.npy', )
yVal1true_Original1 = cp.load('yVal1true_Original1.npy', )

In [19]:
yTrain1final_g2.shape

(18360, 16, 20)

In [20]:
yVal1final_g1.shape

(2040, 16, 20)

In [ ]:
#yTrain1true_Original1 = yTrain1true_Original1[68:, :, :]

In [ ]:
yTrain1true_Original1.shape, yVal1final_g1.shape

In [ ]:
list1 = list(range(xTrain1final_g2.shape[0]))
list2 = list(range(xVal1final_g1.shape[0]))

In [ ]:
list2

In [ ]:
from keras.regularizers import l1, l2
from keras.layers import Conv1D, LSTM
def bilstm_attention(input_shape,bi_layers,optim, lstm_units,lstm_act,
                     return_seq,lstm_kernel_type,dropout_percent,
                     dense_units,dense_act,loss_metrics, forecast_window, rolling_window1,
                     custom_loss3, weights_tf, A_tensor, data_max, mag_A_inverse_tensor):
    
    input_layer = Input(shape=input_shape)
    print("Shape of input_layer:", input_layer.shape)
    #layer = Dense(dense_units,activation = dense_act)(input_layer)
    #layer = Conv1D(filters=32, kernel_size=3)(layer)
    layer = TimeDistributed(tf.keras.layers.Flatten())(input_layer)
    for i in range(0, bi_layers):
        layer = Bidirectional(LSTM(lstm_units,activation=lstm_act,return_sequences=return_seq,
                                   kernel_initializer=lstm_kernel_type,))(layer)
        print("Shape of BILSTM_layer:", layer.shape)
        attention_layer = attention()(layer)
        layer = tf.keras.layers.Dropout(dropout_percent)(layer)
    #layer 3
    flat_layer = Flatten()(layer)
    print("Shape of flat_layer:", layer.shape)
    reshape_layer = Reshape((forecast_window,2*(lstm_units*(int(rolling_window1/forecast_window)))))(flat_layer)
    print("Shape of reshaped_layer:", reshape_layer.shape)
    layer = Dense(dense_units,activation = dense_act)(reshape_layer)
    output_layer = Dense(input_shape[1], bias_regularizer = l2(0.1), kernel_regularizer = l2(0.0001), )(layer) #
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    #optim = tf.keras.optimizers.Adam(learning_rate=lr_schedule(step),decay = 0,clipnorm=None)
    model.compile(loss=custom_loss5(weights_tf = weights_tf, A_tensor= A_tensor, data_max = data_max, mag_A_inverse_tensor = mag_A_inverse_tensor),metrics=loss_metrics, optimizer= optim)
    model.summary()
    return model

In [ ]:
int(xTrain1final_g2.shape[0]/68) * 10

list1 = []
for j in range (0, int(xTrain1final_g2.shape[0]/68)):
    '''
    for i in range (8, 17):
        list1.append(j * 68 + i)
    '''
    for i in range (42, 52):
        list1.append(j * 68 + i)

In [ ]:
max(list1), xTrain1final_g2.shape

list2 = []
for j in range (0, int(xVal1final_g1.shape[0]/68)):

    for i in range (8, 17):
        list2.append(j * 68 + i)

    for i in range (42, 52):
        list2.append(j * 68 + i)

In [ ]:
xTrain1final_g2.shape, xTrain1final_g2.shape

In [ ]:
xTrain1 = xTrain1final_g2[list1, 12-rolling_window1:, :].get()#/ data_max_mag * data_max
yTrain1 = yTrain1final_g2[list1, 12:, :].get()#/ data_max_mag * data_max
xVal1 = xVal1final_g1[list2,12-rolling_window1:, :].get()#/ data_max_mag * data_max
yVal1 = yVal1final_g1[list2, 12:, :].get()#/ data_max_mag * data_max

print(xTrain1.shape,yTrain1.shape, xVal1.shape, yVal1.shape)
yVal1true_Original1.shape, yTrain1true_Original1.shape

In [ ]:
augmented_x_train = xTrain1.copy()
augmented_y_train = yTrain1.copy()

In [ ]:
def time_warping(sequence, factor_range=(0.8, 1.2)):
    warp_factor = np.random.uniform(factor_range[0], factor_range[1])
    num_points = sequence.shape[0]
    warped_indices = np.arange(0, num_points, warp_factor)
    warped_indices = np.clip(warped_indices, 0, num_points - 1).astype(int)
    return sequence[warped_indices]

# Double the data by applying time warping augmentation



In [ ]:
num_sequences = xTrain1.shape[0]
num_output_elements = 4  # Assuming your output sequences have 4 elements each

# Initialize all weights to 1
weights = np.ones((num_sequences, num_output_elements))

# Define the prioritized ranges
prioritized_ranges_morning = []
prioritized_ranges_evening = []
for i in range(0, num_sequences, 68):
    range1 = list(range(i + 8, i + 17))  # Prioritize 8th to 16th sequence in each range
    range2 = list(range(i + 40, i + 54))  # Prioritize 42nd to 52nd sequence in each range
    prioritized_ranges_morning.extend(range1)
    prioritized_ranges_evening.extend(range2)
'''
prioritized_ranges_morning = [num for num in prioritized_ranges_morning if num <= len(xTrain1)]
prioritized_ranges_evening = [num for num in prioritized_ranges_evening if num <= len(xTrain1)]
# Assign higher weights to the prioritized sequences for each output element
weights[prioritized_ranges_morning, :] = 1
# Assign higher weights to the prioritized sequences for each output element
weights[prioritized_ranges_evening, :] = 1
'''
weights_tf = tf.constant(weights, dtype=tf.float32)

In [ ]:
xVal1.shape[0]/35

In [ ]:
num_time_periods = yTrain1.shape[1]
num_spatial_features = yTrain1.shape[2]
weights_tf = tf.tile(weights[:, :, tf.newaxis], (1, 1, num_spatial_features))  # Shape: (1156, 4, 15)

def custom_loss3(weights_tf):
    def my_loss(y_true, y_pred):
        tf.print(weights_tf.shape, y_true.shape, y_pred.shape)
        squared_diff = tf.square(y_true - y_pred)
        batch_indices = tf.range(batchs)  # Define batch indices
        #tf.print(batch_indices, batchs)
        # Select the corresponding weights for the batch
        tiled_batch_weights = tf.gather(weights_tf, batch_indices)  # Shape: (batch_size, 10, 4)
        '''tf.print(tf.reduce_sum(tiled_batch_weights), 
                 tiled_batch_weights.shape, 
                 tf.reduce_max(tiled_batch_weights))'''
        tiled_batch_weights_float32 = tf.cast(tiled_batch_weights, dtype=tf.float32)
        #tf.print('tiled_batch_weights', tiled_batch_weights.shape)
        #tf.print('tiled_batch_weights_float32', tiled_batch_weights_float32.shape)
        squared_diff_float32 = tf.cast(squared_diff, dtype=tf.float32)
        #tf.print('squared_diff_float32', squared_diff_float32.shape)
        #tf.print(squared_diff_float32, tiled_batch_weights_float32)
        weighted_squared_diff = squared_diff_float32 * tiled_batch_weights_float32
        return weighted_squared_diff
    return my_loss

In [ ]:

y_pred_transposed = tf.transpose(tf.convert_to_tensor(yTrain1, dtype=tf.float32), [2, 1, 0])
output_array_pred = tf.reduce_sum(y_pred_transposed, axis=1)
#data_max * tf.matmul(A_tensor, tf.matmul(mag_A_inverse_tensor, tf.convert_to_tensor(output_array_pred)))

In [ ]:
data_max

In [ ]:
A_tensor = tf.convert_to_tensor(A, dtype=tf.float32)
mag_A_inverse_tensor = tf.convert_to_tensor(mag_A_inverse, dtype=tf.float32)


def custom_loss5(weights_tf, A_tensor, mag_A_inverse_tensor, data_max):
    def my_loss(y_true, y_pred):
        #constant = 1/ (weights_tf.shape[0] * weights_tf.shape[2])
        #tf.print(weights_tf.shape, y_true.shape, y_pred.shape)
        y_pred_transposed = tf.transpose(y_pred, [2, 1, 0])
        y_pred_transposed = tf.maximum(y_pred_transposed, 0.0)
        output_array_pred = tf.reduce_sum(y_pred_transposed, axis=1)
        dot_product_ypred =  data_max * tf.matmul(A_tensor,
                                                  tf.matmul(mag_A_inverse_tensor, tf.convert_to_tensor(output_array_pred)))


        y_true_transposed = tf.transpose(y_true, [2, 1, 0])
        output_array_true = tf.reduce_sum(y_true_transposed, axis=1)
        dot_product_ytrue =  data_max * tf.matmul(A_tensor, 
                                                  tf.matmul(mag_A_inverse_tensor, tf.convert_to_tensor(output_array_true)))
        #tf.print(dot_product_ytrue)
        #tf.print(dot_product_ypred)
        
        neum = 2* tf.square(dot_product_ytrue - dot_product_ypred) + 1
        denom = dot_product_ytrue + tf.abs(dot_product_ypred) + 1
        
        squared_difference = tf.math.sqrt(neum / denom)
        #tf.print('neum' , neum, 'denom', denom, 'squared_difference', squared_difference)
        five_minus_V = 5- squared_difference
        five_minus_V = tf.where(tf.equal(five_minus_V, 0), 0.001, five_minus_V)
        abs_five_minus_V = abs(five_minus_V)
        final = 0.5 -  0.5*(five_minus_V)/(abs_five_minus_V)
        #tf.print(constant * tf.reduce_max(final))
        #tf.print(squared_difference,  tf.reduce_max(squared_difference), tf.reduce_sum(final))
        squared_difference2 = tf.square(y_true - y_pred)
        #tf.print(tf.reduce_sum( squared_difference2) , tf.reduce_sum(final))
        return  tf.reduce_sum( squared_difference2)  #tf.reduce_sum(final) #+ 0.0001* tf.reduce_sum( squared_difference2) 
    return my_loss

In [ ]:
A_tensor = tf.convert_to_tensor(A, dtype=tf.float32)
mag_A_inverse_tensor = tf.convert_to_tensor(mag_A_inverse, dtype=tf.float32)


def custom_loss5(weights_tf, A_tensor, mag_A_inverse_tensor, data_max):
    def my_loss(y_true, y_pred):
        #constant = 1/ (weights_tf.shape[0] * weights_tf.shape[2])
        #tf.print(weights_tf.shape, y_true.shape, y_pred.shape)
        y_pred_transposed = tf.transpose(y_pred, [2, 1, 0])
        y_pred_transposed = tf.maximum(y_pred_transposed, 0.0)
        output_array_pred = tf.reduce_sum(y_pred_transposed, axis=1)

        y_true_transposed = tf.transpose(y_true, [2, 1, 0])
        output_array_true = tf.reduce_sum(y_true_transposed, axis=1)

        squared_difference2 = tf.square(y_true - y_pred)
        difference_hourly2 = tf.square(output_array_true - output_array_pred)
        #tf.print(tf.reduce_sum( squared_difference2) , tf.reduce_sum(final))
        return  tf.reduce_sum( squared_difference2) + 0.00001* tf.reduce_sum( difference_hourly2) #tf.reduce_sum(final) #+ 0.0001* tf.reduce_sum( squared_difference2) 
    return my_loss

In [ ]:
def custom_loss(alpha = 0.1):
    def my_loss_fn_GEH_percent(y_true, y_pred):
        squared_difference = tf.math.sqrt(2 * tf.square(4*y_true - 4*y_pred) + 0.000000001)/(4* y_true + tf.abs(4*y_pred) +  0.000000001)
        five_minus_V = 5- squared_difference
        abs_five_minus_V = abs(5- squared_difference)
        final = 0.5 +  0.5*five_minus_V/abs_five_minus_V
        squared_difference2 = tf.square(y_true - y_pred)
        
        return tf.reduce_sum(0* final + squared_difference2)
    return my_loss_fn_GEH_percent

In [ ]:
type(A)

In [ ]:
import tensorflow as tf

# Limit GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

def bilstm_attention_new1(input_shape,bi_layers,optim, lstm_units,lstm_act,
                     return_seq,lstm_kernel_type,dropout_percent,
                     dense_units,dense_act,loss_metrics, forecast_window, rolling_window1, custom_loss3, weights_tf):
    print('yes')
    input_layer = Input(shape=input_shape)
    tf.print("Shape of input_layer:", input_layer.shape)
    layer = Reshape((rolling_window1, input_shape[0] *input_shape[2]  ))(input_layer)
    '''layer = Flatten(input_shape=(input_shape[0], rolling_window1, input_shape[2]))(input_layer)
    tf.print('Flatten', layer.shape)
    layer = Dense(rolling_window1 * input_shape[-1], activation='elu',
                    bias_regularizer=regularizers.L2(.1), 
                    kernel_regularizer=regularizers.L2(.1),)(layer)
    tf.print('Dense', layer.shape)
    layer = Reshape((rolling_window1, input_shape[-1]))(layer)
    tf.print('Rehape', layer.shape)'''
    layer = TimeDistributed(tf.keras.layers.Flatten())(layer)
    tf.print('flattened layer', layer.shape)
    for i in range(0, bi_layers):
        layer = Bidirectional(LSTM(lstm_units,activation=lstm_act,return_sequences=return_seq,
                                   kernel_initializer=lstm_kernel_type,
                                   bias_regularizer=regularizers.L2(.00001), 
                                   kernel_regularizer=regularizers.L2(.00001),))(layer)
        print("Shape of BILSTM_layer:", layer.shape)
        attention_layer = attention()(layer)
        layer = tf.keras.layers.Dropout(dropout_percent)(layer)
    #layer 3
    flat_layer = Flatten()(layer)
    print("Shape of flat_layer:", layer.shape)
    reshape_layer = Reshape((forecast_window,2*(lstm_units*(int(rolling_window1/forecast_window)))))(flat_layer)
    print("Shape of reshaped_layer:", reshape_layer.shape)
    print('dense_units', dense_units)
    layer = Dense(dense_units,activation = dense_act)(reshape_layer)
    print('dense_layer', layer.shape)
    output_layer = Dense(input_shape[2],bias_regularizer=regularizers.L2(.01))(layer)
    print('output_layer', layer.shape)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    #optim = tf.keras.optimizers.Adam(learning_rate=lr_schedule(step),decay = 0,clipnorm=None)
    model.compile(loss=custom_loss3(weights_tf = weights_tf),metrics=loss_metrics, optimizer= optim)
    model.summary()
    return model

In [ ]:
from keras.utils import Sequence
class SequenceDataGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size, data_augmentation=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.data_augmentation = data_augmentation

    def __len__(self):
        return int(np.ceil(len(self.x_data) / self.batch_size))

    def __getitem__(self, index):
        batch_x = self.x_data[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y_data[index * self.batch_size:(index + 1) * self.batch_size]

        if self.data_augmentation:
            # Apply your data augmentation transformations here
            pass

        return batch_x, batch_y

In [ ]:
data_generator = SequenceDataGenerator(xTrain1, yTrain1, batch_size=batchs)

In [ ]:
data_generator

In [ ]:
from tensorflow.keras import regularizers
from timeit import default_timer as timer

input_shape=(xTrain1.shape[1],xTrain1.shape[2])
print(xTrain1.shape[1],yTrain1.shape[2])
model_bilstm2 = bilstm_attention(input_shape,bi_layers,optim, lstm_units,lstm_act,
                                return_seq,lstm_kernel_type,dropout_percent,
                                dense_units,dense_act,loss_metrics, forecast_window, rolling_window1,
                                custom_loss5, weights_tf, A_tensor, data_max, mag_A_inverse_tensor)
model_history = model_bilstm2.fit(xTrain1,yTrain1,
                           validation_data = (xVal1,yVal1),
                           epochs=Epochs,callbacks=clbcks,batch_size=batchs,shuffle=True)
print('Total Model Training Time >> ',sum(cb.logs)/60)

In [ ]:
model_bilstm2.summary()

In [ ]:
from livelossplot import PlotLossesKeras
#batchs = 68*270
Epochs = 100
optim = tf.keras.optimizers.Adam(learning_rate=0.00001)
lr_metric = get_lr_metric(optim)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      mode='min', 
                                      patience=500)
cb = TimingCallback()
clbcks = [es,cb, PlotLossesKeras(from_step=1)]
loss_metrics = [RootMeanSquaredError(name='rmse'),
                MeanAbsolutePercentageError(name='mape'),
                get_lr_metric(optim)]

In [ ]:
model_bilstm2.compile(loss=custom_loss5(weights_tf = weights_tf, A_tensor = A_tensor,data_max = data_max, mag_A_inverse_tensor = mag_A_inverse_tensor),
                      metrics=loss_metrics, 
                      optimizer= optim)
model_history = model_bilstm2.fit(xTrain1,yTrain1,
                           validation_data = (xVal1,yVal1),
                           epochs=Epochs,callbacks = clbcks,
                                  batch_size=batchs,shuffle=True)

1. Training 

In [ ]:
Index_final = Index_final.tolist()
list1, len(Index_final)

In [ ]:
#xTrain1_Original =xTrain1true_Original1[list1, :, :]
yTrain1_Original = yTrain1true_Original1[list1, :, :]
#xTrain1_Original  = xTrain1true_Original1 [list1, :, :]
yTrain1_Original  = yTrain1true_Original1 [list1, :, :]
#multiindex = pd.MultiIndex.from_tuples(Index_final[])
multiindex = pd.MultiIndex.from_tuples([Index_final[position] for position in list1])
multiindex = multiindex
train_pred = model_bilstm2.predict(xTrain1)
train_truth = yTrain1
#df_true = pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(yTrain1, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df_pred = pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(train_pred, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df = df_pred.copy(deep = True)
sums_df_pred = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_pred.columns = multiindex
sums_df_pred[sums_df_pred<0] = 0
df_rec= pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(yTrain1, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df = df_rec.copy(deep = True)
sums_df_rec = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_rec.columns = multiindex
sums_df_rec[sums_df_rec<0] = 0
df_true_original = pd.DataFrame(unfolding_3D(yTrain1true_Original1.get()[list1, :, :], unfol_dim = 2, other_dim_seq = [0,1] ))
df = df_true_original.copy()
sums_df_Original = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_Original.columns = multiindex
GEH_df = np.sqrt((2* np.power(sums_df_Original - sums_df_pred, 2) + 0.00000001)/ (sums_df_Original + sums_df_pred + 0.00000001))
print((np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001)).isna().any().any())
gedf = (np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001))
print(sums_df_pred[sums_df_pred<0].any().any())
print((GEH_df[GEH_df<=5] / GEH_df[GEH_df<=5]).sum().sum() / (GEH_df.shape[0]* GEH_df.shape[1]) * 100)
GEH_df_time_of_day = GEH_df.T.unstack(0).T
less_than_5_count = (GEH_df_time_of_day <= 5).sum(axis = 0)
# Calculate the total number of values in the 'Column1' column
total_count = GEH_df_time_of_day.count(axis = 0)
# Compute the percentage of values less than 5
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.plot()
print('Morning average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[28:36].mean()[0])
print('Evening average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[62:72].mean()[0])

print('Motorways')
sums_df_Original = sums_df_Original.T
sums_df_Original.columns = train_data_shubham.columns
sums_df_pred = sums_df_pred.T
sums_df_pred.columns = train_data_shubham.columns
sums_df_pred = sums_df_pred[MWY_Sections['Sections'].to_list()].T
sums_df_Original = sums_df_Original[MWY_Sections['Sections'].to_list()].T
GEH_df = np.sqrt((2* np.power(sums_df_Original - sums_df_pred, 2) + 0.00000001)/ (sums_df_Original 
                                                                                  + sums_df_pred + 0.00000001))
print((np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001)).isna().any().any())
gedf = (np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001))
print(sums_df_pred[sums_df_pred<0].any().any())
print((GEH_df[GEH_df<=5] / GEH_df[GEH_df<=5]).sum().sum() / (GEH_df.shape[0]* GEH_df.shape[1]) * 100)
GEH_df_time_of_day = GEH_df.T.unstack(0).T
less_than_5_count = (GEH_df_time_of_day <= 5).sum(axis = 0)
# Calculate the total number of values in the 'Column1' column
total_count = GEH_df_time_of_day.count(axis = 0)
# Compute the percentage of values less than 5
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.plot()
print('Morning average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[28:36].mean()[0])
print('Evening average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[62:72].mean()[0])

In [ ]:
#xTrain1_Original =xTrain1true_Original1[list2, :, :]
yTrain1_Original = yTrain1true_Original1[list2, :, :]
#xTrain1_Original  = xTrain1true_Original1 [list2, :, :]
yTrain1_Original  = yTrain1true_Original1 [list2, :, :]
#multiindex = pd.MultiIndex.from_tuples(Index_final[])
multiindex = pd.MultiIndex.from_tuples([Index_final[position] for position in list2])
multiindex = multiindex
train_pred = model_bilstm2.predict(xVal1)
train_truth = yTrain1

In [ ]:

#df_true = pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(yTrain1, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df_pred = pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(train_pred, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df = df_pred.copy(deep = True)
sums_df_pred = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_pred.columns = multiindex
sums_df_pred[sums_df_pred<0] = 0
df_rec= pd.DataFrame(A.dot(mag_A_inverse.dot(unfolding_3D(yVal1, unfol_dim = 2, other_dim_seq = [0,1] ))))*data_max #/data_max * data_max_mag 
df = df_rec.copy(deep = True)
sums_df_rec = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_rec.columns = multiindex
sums_df_rec[sums_df_rec<0] = 0
df_true_original = pd.DataFrame(unfolding_3D(yVal1true_Original1.get()[list2, :, :], unfol_dim = 2, other_dim_seq = [0,1] ))
df = df_true_original.copy()
sums_df_Original = pd.DataFrame({
    f'Sum_{i//4+1}': df.iloc[:, i:i+4].sum(axis=1)
    for i in range(0, len(df.columns), 4)
})
# Set the MultiIndex as the column names of the sums_df DataFrame
sums_df_Original.columns = multiindex
GEH_df = np.sqrt((2* np.power(sums_df_Original - sums_df_pred, 2) + 0.00000001)/ (sums_df_Original + sums_df_pred + 0.00000001))
print((np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001)).isna().any().any())
gedf = (np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001))
print(sums_df_pred[sums_df_pred<0].any().any())
print((GEH_df[GEH_df<=5] / GEH_df[GEH_df<=5]).sum().sum() / (GEH_df.shape[0]* GEH_df.shape[1]) * 100)
GEH_df_time_of_day = GEH_df.T.unstack(0).T
less_than_5_count = (GEH_df_time_of_day <= 5).sum(axis = 0)
# Calculate the total number of values in the 'Column1' column
total_count = GEH_df_time_of_day.count(axis = 0)
# Compute the percentage of values less than 5
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.plot()
print('Morning average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[28:36].mean()[0])
print('Evening average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[62:72].mean()[0])

print('Motorways')
sums_df_Original = sums_df_Original.T
sums_df_Original.columns = train_data_shubham.columns
sums_df_pred = sums_df_pred.T
sums_df_pred.columns = train_data_shubham.columns
sums_df_pred = sums_df_pred[MWY_Sections['Sections'].to_list()].T
sums_df_Original = sums_df_Original[MWY_Sections['Sections'].to_list()].T
sums_df_rec = sums_df_rec.T
sums_df_rec.columns = train_data_shubham.columns
sums_df_rec = sums_df_rec[MWY_Sections['Sections'].to_list()].T
GEH_df = np.sqrt((2* np.power(sums_df_Original - sums_df_pred, 2) + 0.00000001)/ (sums_df_Original + sums_df_pred + 0.00000001))
print((np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001)).isna().any().any())
gedf = (np.power(sums_df_Original - sums_df_pred, 2)/ (sums_df_Original + sums_df_pred + 0.00000001))
print(sums_df_pred[sums_df_pred<0].any().any())
print((GEH_df[GEH_df<=5] / GEH_df[GEH_df<=5]).sum().sum() / (GEH_df.shape[0]* GEH_df.shape[1]) * 100)
GEH_df_time_of_day = GEH_df.T.unstack(0).T
less_than_5_count = (GEH_df_time_of_day <= 5).sum(axis = 0)
# Calculate the total number of values in the 'Column1' column
total_count = GEH_df_time_of_day.count(axis = 0)
# Compute the percentage of values less than 5
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.plot()
print('Morning average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[28:36].mean()[0])
print('Evening average' , pd.DataFrame(percentage_less_than_5_time_of_day).loc[64:72].mean()[0])


In [ ]:
pd.DataFrame(A)

In [ ]:
GEH_df_time_of_day_eve = GEH_df_time_of_day.T[44:52].T.unstack(1)
less_than_5_count = (GEH_df_time_of_day_eve <= 5).sum(axis = 1)
total_count = GEH_df_time_of_day_eve.count(axis = 1)
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.sort_values()

In [ ]:
GEH_df_time_of_day_eve = GEH_df_time_of_day.T[44:52].T.unstack(0)
less_than_5_count = (GEH_df_time_of_day_eve <= 5).sum(axis = 1)
total_count = GEH_df_time_of_day_eve.count(axis = 1)
percentage_less_than_5_time_of_day = (less_than_5_count / total_count) * 100
percentage_less_than_5_time_of_day.sort_values()

In [ ]:
df = GEH_df_time_of_day_eve.copy(deep = True)
# Define the bins
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, float('inf')]

# Calculate the percentage of values in each bin
bin_counts = pd.cut(df.unstack(1), bins=bins, right=False).value_counts(sort=False)
total_count = len(df.unstack(1))

percentage_values = (bin_counts / total_count) * 100

# Print the results
for bin_range, percentage in zip(bin_counts.index, percentage_values):
    print(f'Range {bin_range}: {percentage:.2f}%')

In [ ]:
22.74+20.11+16.20+13.20+9.61

In [ ]:
df = df_pred.copy(deep = True)
valid_times1 = ['00:00:00', '00:15:00', '00:30:00', '00:45:00', '01:00:00', '01:15:00',
       '01:30:00', '01:45:00', '02:00:00', '02:15:00', '02:30:00', '02:45:00',
       '03:00:00', '03:15:00', '03:30:00', '03:45:00', '04:00:00', '04:15:00',
       '04:30:00',  '21:45:00', '22:00:00', '22:15:00',
       '22:30:00', '22:45:00', '23:00:00', '23:15:00', '23:30:00', '23:45:00']
valid_times2 = ['00:00:00', '00:15:00', '00:30:00', '00:45:00', '01:00:00', '01:15:00',
       '01:30:00', '01:45:00', '02:00:00', '02:15:00', '02:30:00', '02:45:00',
       '03:00:00', '03:15:00', '03:30:00', '03:45:00', '04:00:00', '04:15:00',
       '04:30:00',  '21:45:00', '22:00:00', '22:15:00',
       '22:30:00', '22:45:00', '23:00:00', '23:15:00', '23:30:00', '23:45:00']
valid_times3 = ['00:00:00', '00:15:00', '00:30:00', '00:45:00', '01:00:00', '01:15:00',
       '01:30:00', '01:45:00', '02:00:00', '02:15:00', '02:30:00', '02:45:00',
       '03:00:00', '03:15:00', '03:30:00', '03:45:00', '04:00:00', '04:15:00',
       '04:30:00',  '21:45:00', '22:00:00', '22:15:00',
       '22:30:00', '22:45:00', '23:00:00', '23:15:00', '23:30:00', '23:45:00']
valid_times4 = ['00:00:00', '00:15:00', '00:30:00', '00:45:00', '01:00:00', '01:15:00',
       '01:30:00', '01:45:00', '02:00:00', '02:15:00', '02:30:00', '02:45:00',
       '03:00:00', '03:15:00', '03:30:00', '03:45:00', '04:00:00', '04:15:00',
       '04:30:00',  '21:45:00', '22:00:00', '22:15:00',
       '22:30:00', '22:45:00', '23:00:00', '23:15:00', '23:30:00', '23:45:00']

In [ ]:
df_pred

In [ ]:
df = df_pred.copy(deep = True)
dataframes = []
for i in range(4):
    selected_cols = df.iloc[:, i::4]
    dataframes.append(selected_cols)
# Rename the columns of the split DataFrames
for i, selected_df in enumerate(dataframes):
    selected_df.columns = [f'col{i*4+j}' for j in range(selected_df.shape[1])]

In [ ]:
dataframes[0]

In [ ]:
df = df_true_original.copy(deep = True)
dataframes_true = []
for i in range(4):
    selected_cols = df.iloc[:, i::4]
    dataframes_true.append(selected_cols)

# Rename the columns of the split DataFrames
for i, selected_df in enumerate(dataframes_true):
    selected_df.columns = [f'col{i*4+j}' for j in range(selected_df.shape[1])]

In [ ]:
df = df_rec.copy(deep = True)
dataframes_rec = []
for i in range(4):
    selected_cols = df.iloc[:, i::4]
    dataframes_rec.append(selected_cols)

# Rename the columns of the split DataFrames
for i, selected_df in enumerate(dataframes_rec):
    selected_df.columns = [f'col{i*4+j}' for j in range(selected_df.shape[1])]

In [ ]:
test1 = test_data_prior.T.unstack(0)
test1 = test1[~test1.index.isin(valid_times1)].T.unstack(1).T.unstack(0).T.unstack(1)

test2 = test_data_prior.T.unstack(0)
test2 = test2[~test2.index.isin(valid_times1)].T.unstack(1).T.unstack(0).T.unstack(1)

test3 = test_data_prior.T.unstack(0)
test3 = test3[~test3.index.isin(valid_times1)].T.unstack(1).T.unstack(0).T.unstack(1)

test4 = test_data_prior.T.unstack(0)
test4 = test4[~test4.index.isin(valid_times1)].T.unstack(1).T.unstack(0).T.unstack(1)

In [ ]:
test_data_prior.T.unstack(0)[20+42:20+52]

In [ ]:
dataframes[0].index = test1.index
dataframes[0].columns = test1.columns[:dataframes[0].columns.shape[0], ]

dataframes[1].index = test2.index
dataframes[1].columns = test2.columns[:dataframes[0].columns.shape[0], ]

dataframes[2].index = test3.index
dataframes[2].columns = test3.columns[:dataframes[0].columns.shape[0], ]

dataframes[3].index = test4.index
dataframes[3].columns = test4.columns[:dataframes[0].columns.shape[0], ]


dataframes_true[0].index = test1.index
dataframes_true[0].columns = test1.columns[:dataframes[0].columns.shape[0], ]

dataframes_true[1].index = test2.index
dataframes_true[1].columns = test2.columns[:dataframes[0].columns.shape[0], ]

dataframes_true[2].index = test3.index
dataframes_true[2].columns = test3.columns[:dataframes[0].columns.shape[0], ]

dataframes_true[3].index = test4.index
dataframes_true[3].columns = test4.columns[:dataframes[0].columns.shape[0], ]

dataframes_rec[0].index = test1.index
dataframes_rec[0].columns = test1.columns[:dataframes[0].columns.shape[0], ]

dataframes_rec[1].index = test2.index
dataframes_rec[1].columns = test2.columns[:dataframes[0].columns.shape[0], ]

dataframes_rec[2].index = test3.index
dataframes_rec[2].columns = test3.columns[:dataframes[0].columns.shape[0], ]

dataframes_rec[3].index = test4.index
dataframes_rec[3].columns = test4.columns[:dataframes[0].columns.shape[0], ]

In [ ]:
dataframes[0]

In [ ]:
a = 0*68 
b = 3*68
Link = '16671173_st0'
fig, ax = plt.subplots(figsize = (20, 5))
dataframes[0].T[Link].iloc[a:b].plot(color = 'green')
dataframes_true[0].T[Link].iloc[a:b].plot(color = 'red')
dataframes_rec[0].T[Link].iloc[a:b].plot(color = 'black')

In [ ]:
a = 0*68 
b = 10*68
Link = '16671173_st0'
fig, ax = plt.subplots(figsize = (20, 5))
dataframes[1].T[Link].iloc[a:b].plot(color = 'green')
dataframes_true[1].T[Link].iloc[a:b].plot(color = 'red')
dataframes_rec[1].T[Link].iloc[a:b].plot(color = 'black')

In [ ]:
a = 0*68 
b = 5*68
Link = '16671173_st0'
fig, ax = plt.subplots(figsize = (20, 5))
dataframes[2].T[Link].iloc[a:b].plot(color = 'green')
dataframes_true[2].T[Link].iloc[a:b].plot(color = 'red')
dataframes_rec[2].T[Link].iloc[a:b].plot(color = 'black')

In [ ]:
a = 0*68 
b = 5*68
Link = '16670948_st0'
fig, ax = plt.subplots(figsize = (20, 5))
dataframes[3].T[Link].iloc[a:b].plot(color = 'green')
dataframes_true[3].T[Link].iloc[a:b].plot(color = 'red')
dataframes_rec[3].T[Link].iloc[a:b].plot(color = 'black')

In [ ]:
def calculate_rmse_mape(true_df,pred_df, plot = False ):
    rmse = np.sqrt(np.power(true_df - pred_df, 2).sum().sum() / (true_df.shape[0] * true_df.shape[1]))
    rmse_rows = np.sqrt(np.power(true_df.T.unstack(0) - pred_df.T.unstack(0), 2).sum(axis = 1) / (pred_df.T.unstack(0).shape[1]))
    if plot == True:
        rmse_rows.plot()
        plt.show()
    W = (true_df/true_df).replace(np.nan, 0).replace(np.inf, 0)
    MAPE = (abs(true_df - pred_df)/ true_df).replace(np.nan, 0).replace(np.inf, 0).sum().sum() / W.sum().sum() * 100
    MAPE_rows = (abs(true_df.T.unstack(0) - pred_df.T.unstack(0) )/ true_df.T.unstack(0) ).replace(np.nan, 0).replace(np.inf, 0).sum(axis = 1) / W.T.unstack(0).sum(axis = 1) * 100
    if plot == True:
        MAPE_rows.plot()
        plt.show()
    return rmse, rmse_rows, MAPE, MAPE_rows

In [ ]:
pred_df = dataframes[0]
true_df = dataframes_true[0]
rmse, rmse_rows, MAPE, MAPE_rows = calculate_rmse_mape(true_df, pred_df)
rmse_m, rmse_rows_m, MAPE_m, MAPE_rows_m = calculate_rmse_mape(true_df[true_df.index.isin(MWY_Sections['Sections'].to_list())],
                                                               pred_df[true_df.index.isin(MWY_Sections['Sections'].to_list())])
print('15 min ahead', rmse, MAPE)
print('15 min ahead', rmse_m, MAPE_m)
print('Morning stats', MAPE_rows[9:17].mean(), rmse_rows[9:17].mean())
print('Evening stats', MAPE_rows[45:54].mean(), rmse_rows[45:54].mean())
print('Morning stats Motorways', rmse_rows_m[9:17].mean(), MAPE_rows_m[9:17].mean(), )
print('Evening stats Motorways', rmse_rows_m[45:54].mean(), MAPE_rows_m[45:54].mean())

In [ ]:
pred_df = dataframes[1]
true_df = dataframes_true[1]
rmse, rmse_rows, MAPE, MAPE_rows = calculate_rmse_mape(true_df, pred_df)
rmse_m, rmse_rows_m, MAPE_m, MAPE_rows_m = calculate_rmse_mape(true_df[true_df.index.isin(MWY_Sections['Sections'].to_list())],
                                                               pred_df[true_df.index.isin(MWY_Sections['Sections'].to_list())])
print('15 min ahead', rmse, MAPE)
print('15 min ahead', rmse_m, MAPE_m)
print('Morning stats', MAPE_rows[9:17].mean(), rmse_rows[9:17].mean())
print('Evening stats', MAPE_rows[45:54].mean(), rmse_rows[45:54].mean())
print('Morning stats Motorways', rmse_rows_m[9:17].mean(), MAPE_rows_m[9:17].mean(), )
print('Evening stats Motorways', rmse_rows_m[45:54].mean(), MAPE_rows_m[45:54].mean())

In [ ]:
pred_df = dataframes[2]
true_df = dataframes_true[2]
rmse_m, rmse_rows_m, MAPE_m, MAPE_rows_m = calculate_rmse_mape(true_df[true_df.index.isin(MWY_Sections['Sections'].to_list())],
                                                               pred_df[true_df.index.isin(MWY_Sections['Sections'].to_list())])
print('15 min ahead', rmse, MAPE)
print('15 min ahead', rmse_m, MAPE_m)
print('Morning stats', MAPE_rows[9:17].mean(), rmse_rows[9:17].mean())
print('Evening stats', MAPE_rows[45:54].mean(), rmse_rows[45:54].mean())
print('Morning stats Motorways', rmse_rows_m[9:17].mean(), MAPE_rows_m[9:17].mean(), )
print('Evening stats Motorways', rmse_rows_m[45:54].mean(), MAPE_rows_m[45:54].mean())

In [ ]:
pred_df = dataframes[3]
true_df = dataframes_true[3]
rmse_m, rmse_rows_m, MAPE_m, MAPE_rows_m = calculate_rmse_mape(true_df[true_df.index.isin(MWY_Sections['Sections'].to_list())],
                                                               pred_df[true_df.index.isin(MWY_Sections['Sections'].to_list())])
print('15 min ahead', rmse, MAPE)
print('15 min ahead', rmse_m, MAPE_m)
print('Morning stats', MAPE_rows[9:17].mean(), rmse_rows[9:17].mean())
print('Evening stats', MAPE_rows[45:54].mean(), rmse_rows[45:54].mean())
print('Morning stats Motorways', rmse_rows_m[9:17].mean(), MAPE_rows_m[9:17].mean(), )
print('Evening stats Motorways', rmse_rows_m[45:54].mean(), MAPE_rows_m[45:54].mean())

In [ ]:
(true_df - pred_df)